**Install and import the required packages**

In [ ]:
!pip install pyspark
!pip install keras-core

In [ ]:
import librosa
import keras_core
import torch
from IPython.display import Markdown, HTML
import IPython.display as ipd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import pyspark
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
import os
data_dir = "/content/gdrive/My Drive/Colab Notebooks/Data/genres_original"
genre_dirs = [x[0] for x in os.walk(data_dir)]

filepath_dict = {}
for genre_dir in genre_dirs[1:]:
    filepath_list = []
    for fn in os.listdir(genre_dir):
      filepath_list.append(os.path.join(genre_dir, fn))

    filepath_dict[genre_dir.split('/')[-1]] = filepath_list

filepath_dict.keys()

**Display sample audio and waveform of each genre**

In [ ]:
for key in filepath_dict.keys():
  #extract the audio sample
  y, sr = librosa.load(filepath_dict[key][0])
  print(f"{key} sampling rate is: {sr}")
  display(HTML(f"<h3 style='text-align: left'>Audio for {key}</h3>\n"))
  audio_widget = ipd.Audio(data=y, rate=sr)
  display(audio_widget)

  #extract waveform plot
  fig_waveform, ax_waveform = plt.subplots(figsize=(11.23, 3))
  librosa.display.waveshow(y, sr=sr, ax=ax_waveform)
  ax_waveform.set(title='{} sample waveform'.format(key), xlabel='Time (s)',
                  ylabel='Amplitude', xlim=[0, len(y)/sr])
  ax_waveform.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
  plt.show()

  #extract STFT spectrogram and plot in log-scale
  fig_spectrogram, ax_spectrogram = plt.subplots(figsize=(14, 5))  # Slightly wider figure for the spectrogram
  FRAME_SIZE = 1024
  HOP_SIZE = 5201
  D = librosa.stft(y, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
  S, phase = librosa.magphase(D)
  S_db = librosa.amplitude_to_db(S, ref=np.max)
  img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log',
                                 hop_length=HOP_SIZE, ax=ax_spectrogram)
  ax_spectrogram.set(title='{} STFT Spectrogram'.format(key),
                     xlabel='Time (s)', ylabel='Frequency (Hz)')
  fig_spectrogram.colorbar(img, ax=ax_spectrogram, format='%+1.0f dB')  # Add colorbar to the spectrogram figure
  plt.show()
  print(200*"-")

Implement the serial attention model (SAM)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiGRUEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(BiGRUEncoder, self).__init__()
        self.bigru = nn.GRU(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)

    def forward(self, x):
        outputs, _ = self.bigru(x)  # outputs shape: (batch_size, sequence_length, 2 * hidden_dim)
        return outputs

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention_vector = nn.Parameter(torch.randn(hidden_dim * 2, 1))

    def forward(self, encoder_outputs):
        # Linear transformation using a trainable attention vector
        attention_scores = torch.matmul(encoder_outputs, self.attention_vector)  # shape: (batch_size, sequence_length, 1)
        attention_scores = attention_scores.squeeze(-1)  # shape: (batch_size, sequence_length)
        attention_weights = F.softmax(attention_scores, dim=1)  # shape: (batch_size, sequence_length)
        return attention_weights

class Classifier(nn.Module):
    def __init__(self, hidden_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, weighted_representation):
        logits = self.fc(weighted_representation)
        return logits

class SAMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_classes):
        super(SAMModel, self).__init__()
        self.encoder = BiGRUEncoder(input_dim, hidden_dim, num_layers, dropout)
        self.attention = Attention(hidden_dim)
        self.classifier = Classifier(hidden_dim, num_classes)

    def forward(self, x):
        encoder_outputs = self.encoder(x)
        attention_weights = self.attention(encoder_outputs)
        weighted_representation = torch.sum(encoder_outputs * attention_weights.unsqueeze(-1), dim=1)
        logits = self.classifier(weighted_representation)
        return logits

# Example of model initialization and forward pass
input_dim = 128  # Example feature size for each timestep
hidden_dim = 256  # Size of GRU hidden states
num_layers = 2  # Number of stacked GRU layers
dropout = 0.5  # Dropout between RNN layers
num_classes = 10  # Example number of output classes (e.g., genres)

model = SAMModel(input_dim, hidden_dim, num_layers, dropout, num_classes)

# Example spectrogram input (batch_size, sequence_length, input_dim)
spectrogram_input = torch.randn(32, 100, input_dim)

# Forward pass
logits = model(spectrogram_input)


In [ ]:
# Adjusting the SAM model
class SAM(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=True):
        super(SAM, self).__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            bidirectional=bidirectional
        )
        self.attention_weights = nn.Parameter(torch.randn(2 * hidden_size if bidirectional else hidden_size, 1))

    def forward(self, x):
        gru_out, _ = self.gru(x)  # GRU output shape: (batch, seq_length, num_directions * hidden_size)
        attention_scores = torch.matmul(gru_out, self.attention_weights).squeeze(2)  # Shape: (batch, seq_length)
        attention_probs = F.softmax(attention_scores, dim=1).unsqueeze(2)  # Shape: (batch, seq_length, 1)
        weighted_gru_out = gru_out * attention_probs  # Shape: (batch, seq_length, num_directions * hidden_size)
        attended_representation = weighted_gru_out.sum(dim=1)  # Shape: (batch, num_directions * hidden_size)
        return attended_representation

# Assuming input size is 513 based on the input shape of the spectrogram (513 features for each time step)
sam_model = SAM(input_size=513, hidden_size=256)


In [ ]:
class PCNNA(nn.Module):
    def __init__(self):
        super(PCNNA, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(kernel_size=4, stride=4)
        self.conv5 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        self.pool5 = nn.MaxPool2d(kernel_size=4, stride=4)
        self.flatten = nn.Flatten()

        # This will hold the linear layer, once we know the input dimensions
        self.fc = None

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)
        x = self.flatten(x)

        # Dynamically create the fully connected layer if it's not already defined
        if self.fc is None:
            # Assuming the dense layer is meant to output a vector of size equal to the flattened features
            T = x.shape[1]
            self.fc = nn.Linear(T, T).to(x.device)

        attention_scores = self.fc(x)
        return attention_scores

# Now we can instantiate the PAM model without specifying T
pam_model = PCNNA()

# To deal with different input sizes, simply call the forward method
# The first time you call it, it will automatically adjust the fully connected layer
dummy_input = torch.randn(1, 1, 513, 128)  # Replace with actual dimensions
pam_output = pam_model(dummy_input)
